# Tutorial 7: Training a Model

## Training a Sequence Labeling Model

In [5]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from typing import List

# 1. get the corpus
corpus: TaggedCorpus = NLPTaskDataFetcher.load_corpus(NLPTask.CONLL_03_DUTCH).downsample(0.1)
print(corpus)

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print("\ntag_dictionary.idx2item:", tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=3) # default: 150

# 8. plot training curves (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves('resources/taggers/example-ner/loss.tsv')
plotter.plot_weights('resources/taggers/example-ner/weights.txt')

2019-04-29 18:55:10,002 Reading data from /home/wohlg/.flair/datasets/conll_03_dutch
2019-04-29 18:55:10,004 Train: /home/wohlg/.flair/datasets/conll_03_dutch/ned.train
2019-04-29 18:55:10,005 Dev: /home/wohlg/.flair/datasets/conll_03_dutch/ned.testa
2019-04-29 18:55:10,006 Test: /home/wohlg/.flair/datasets/conll_03_dutch/ned.testb
2019-04-29 18:55:10,009 UTF-8 can't read: /home/wohlg/.flair/datasets/conll_03_dutch/ned.train ... using "latin-1" instead.
2019-04-29 18:55:13,141 UTF-8 can't read: /home/wohlg/.flair/datasets/conll_03_dutch/ned.testb ... using "latin-1" instead.
2019-04-29 18:55:14,167 UTF-8 can't read: /home/wohlg/.flair/datasets/conll_03_dutch/ned.testa ... using "latin-1" instead.
TaggedCorpus: 1581 train + 290 dev + 520 test sentences

tag_dictionary.idx2item: [b'<unk>', b'O', b'S-ORG', b'B-PER', b'E-PER', b'S-PER', b'S-MISC', b'S-LOC', b'B-LOC', b'E-LOC', b'B-MISC', b'I-MISC', b'E-MISC', b'B-ORG', b'I-ORG', b'E-ORG', b'I-PER', b'I-LOC', b'<START>', b'<STOP>']
2019-04-

### predict the class of new sentences.

In [11]:
from flair.data import Sentence

# load the model you trained
model = SequenceTagger.load_from_file('resources/taggers/example-ner/final-model.pt')

# create example sentence
sentence = Sentence('Bart loopt op straat voor Amsterdam . Hij ontmoet Tom .')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

2019-04-29 19:01:55,863 loading file resources/taggers/example-ner/final-model.pt
Bart <B-PER> loopt <E-PER> op straat voor Amsterdam . Hij ontmoet Tom .


## Training a Text Classification Model


### corpus loading -- from Tutorial 6

In [12]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.data import TaggedCorpus
from pathlib import Path

# use your own data path
data_folder = Path('/home/wohlg/itmo/misc/cooking_classification/preprocessed')

# load corpus containing training, test and dev data
corpus = TaggedCorpus = NLPTaskDataFetcher.load_classification_corpus(data_folder,
                                                                     test_file='cooking.test',
                                                                     dev_file='cooking.valid',
                                                                     train_file='cooking.train').downsample(0.1)
    
print('Done loading')

## look at the corpus:


2019-04-29 19:02:13,219 Reading data from /home/wohlg/itmo/misc/cooking_classification/preprocessed
2019-04-29 19:02:13,226 Train: /home/wohlg/itmo/misc/cooking_classification/preprocessed/cooking.train
2019-04-29 19:02:13,228 Dev: /home/wohlg/itmo/misc/cooking_classification/preprocessed/cooking.valid
2019-04-29 19:02:13,230 Test: /home/wohlg/itmo/misc/cooking_classification/preprocessed/cooking.test
Done loading


In [13]:
label_dict = corpus.make_label_dictionary()

print(corpus.obtain_statistics())


{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1241,
        "number_of_documents_per_class": {
            "sauce": 39,
            "cheese": 24,
            "tea": 18,
            "food-safety": 92,
            "salmon": 6,
            "cookies": 22,
            "texture": 8,
            "american-cuisine": 1,
            "dehydrating": 3,
            "equipment": 84,
            "slow-cooking": 10,
            "crockpot": 5,
            "resources": 6,
            "storage-method": 34,
            "veal": 3,
            "free-range": 2,
            "oven": 26,
            "elderberries": 2,
            "kombucha": 1,
            "sorbet": 1,
            "seafood": 8,
            "bread": 52,
            "dough": 19,
            "yeast": 16,
            "ice-cream": 11,
            "pasta": 24,
            "beets": 3,
            "food-science": 22,
            "cooking-time": 12,
            "pressure-cooker": 3,
            "meat": 35,
           

In [15]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings, DocumentPoolEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


# 1. get the corpus
# corpus: TaggedCorpus = NLPTaskDataFetcher.load_corpus(NLPTask.AG_NEWS, 'path/to/data/folder').downsample(0.1)

# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                   #FlairEmbeddings('news-forward'),
                   #FlairEmbeddings('news-backward'),
                   FlairEmbeddings('news-forward-fast'),
                   FlairEmbeddings('news-backward-fast'),
                   ]
# word_embeddings = [CharacterEmbeddings()]
print('Done Here')


Done Here


In [16]:
print("OK")
word_embeddings

OK


[WordEmbeddings(), FlairEmbeddings(
   (lm): LanguageModel(
     (drop): Dropout(p=0.25)
     (encoder): Embedding(275, 100)
     (rnn): LSTM(100, 1024)
     (decoder): Linear(in_features=1024, out_features=275, bias=True)
   )
 ), FlairEmbeddings(
   (lm): LanguageModel(
     (drop): Dropout(p=0.25)
     (encoder): Embedding(275, 100)
     (rnn): LSTM(100, 1024)
     (decoder): Linear(in_features=1024, out_features=275, bias=True)
   )
 )]

In [24]:
from flair.embeddings import CharacterEmbeddings

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)

#document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
#                                                                     hidden_size=512,
#                                                                     reproject_words=True,
#                                                                     reproject_words_dimension=256,
#                                                                     )
    
print('Start')
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=32,
                                                                     reproject_words=False,
                                                                     reproject_words_dimension=40,
                                                                     dropout=0.25
                                                                     )


    
#document_embeddings= DocumentPoolEmbeddings([CharacterEmbeddings()],
#                                              #flair_embedding_backward,
#                                              #flair_embedding_forward],
#                                              mode='max') # mean (default), min, max

# 5. create the text classifier
#classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False) 
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=True) 
print('Done')



Start
Done


In [19]:
classifier




TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings()
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=2148, bias=True)
    (rnn): RNNBase(2148, 32)
    (dropout): Dropout(p=0.25)
  )
  (decoder): Linear(in_features=32, out_features=512, bias=True)
  (loss_function): BCELoss()
)

In [20]:
# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

# 7. start the training
trainer.train('resources/taggers/cooking',
              learning_rate=0.2, # 0.1 
              mini_batch_size=8, # 32
              anneal_factor=0.5,
              patience=5,
              max_epochs=3) #150
                
# 8. plot training curves (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves('resources/taggers/cooking/loss.tsv')
plotter.plot_weights('resources/taggers/cooking/weights.txt')

2019-04-29 19:03:58,275 ----------------------------------------------------------------------------------------------------
2019-04-29 19:03:58,278 Evaluation method: MICRO_F1_SCORE
2019-04-29 19:03:58,282 ----------------------------------------------------------------------------------------------------
2019-04-29 19:03:58,784 epoch 1 - iter 0/156 - loss 0.08751200
2019-04-29 19:04:07,837 epoch 1 - iter 15/156 - loss 0.08684322
2019-04-29 19:04:16,136 epoch 1 - iter 30/156 - loss 0.08644430
2019-04-29 19:04:24,241 epoch 1 - iter 45/156 - loss 0.08599108
2019-04-29 19:04:31,840 epoch 1 - iter 60/156 - loss 0.08549880
2019-04-29 19:04:40,841 epoch 1 - iter 75/156 - loss 0.08499049
2019-04-29 19:04:45,711 ----------------------------------------------------------------------------------------------------
2019-04-29 19:04:45,712 Exiting from training early.
2019-04-29 19:04:45,714 Saving model ...
2019-04-29 19:04:49,009 Done.
2019-04-29 19:04:49,010 ------------------------------------

2019-04-29 19:04:50,013 brownies   tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,014 brussels-sprouts tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,016 budget-cooking tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,017 bulk-cooking tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,018 butchering tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,019 butter     tp: 0 - fp: 0 - fn: 2 - tn: 148 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,020 buttermilk tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0

2019-04-29 19:04:50,112 condiments tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,113 consistency tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,116 convection tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,120 conversion tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,121 cookbook   tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,123 cookies    tp: 0 - fp: 0 - fn: 4 - tn: 146 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,124 cooking-myth tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
201

2019-04-29 19:04:50,200 experimental tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,201 extracts   tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,202 faq        tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,203 fats       tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,205 fermentation tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,206 filtering  tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,207 fire       tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
20

2019-04-29 19:04:50,278 ham        tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,279 hamburgers tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,281 hard-boiled-eggs tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,282 heat       tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,284 herbs      tp: 0 - fp: 0 - fn: 2 - tn: 148 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,285 high-altitude tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,286 history    tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.00

2019-04-29 19:04:50,360 melon      tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,361 melting    tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,363 melting-chocolate tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,364 menu-planning tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,365 meringue   tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,367 mexican-cuisine tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,369 microwave  tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score

2019-04-29 19:04:50,481 pickling   tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,483 pie        tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,484 pizza      tp: 0 - fp: 0 - fn: 4 - tn: 146 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,486 pizza-stone tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,487 plating    tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,488 please-remove-this-tag tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,489 plums      tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 

2019-04-29 19:04:50,596 salmon     tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,597 salsa      tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,604 salt       tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,606 sandwich   tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,607 sashimi    tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,609 sauce      tp: 0 - fp: 0 - fn: 2 - tn: 148 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,610 sauerkraut tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-0

2019-04-29 19:04:50,687 stock      tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,688 storage    tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,689 storage-lifetime tp: 0 - fp: 0 - fn: 3 - tn: 147 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,689 storage-method tp: 0 - fp: 0 - fn: 1 - tn: 149 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,696 stove      tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,697 strawberries tp: 0 - fp: 0 - fn: 0 - tn: 150 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-04-29 19:04:50,700 substitutions tp: 0 - fp: 0 - fn: 13 - tn: 137 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-scor

2019-04-29 19:04:50,760 ----------------------------------------------------------------------------------------------------
2019-04-29 19:04:50,809 No handles with labels found to put in legend.
2019-04-29 19:04:50,828 No handles with labels found to put in legend.
2019-04-29 19:04:50,852 No handles with labels found to put in legend.


In [21]:
classifier.state_dict

<bound method Module.state_dict of TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings()
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=2148, bias=True)
    (rnn): RNNBase(2148, 32)
    (dropout): Dropout(p=0.25)
  )
  (decoder): Linear(in_features=32, out_features=512, bias=True)
  (loss_function): BCELoss()
)>

In [23]:
from flair.data import Sentence

classifier = TextClassifier.load_from_file('resources/taggers/cooking/final-model.pt')

# create example sentence
sentence = Sentence('where is the bacon, is it in the oven.')

# predict tags and print
classifier.predict(sentence)

print(sentence.labels)

2019-04-29 19:04:56,706 loading file resources/taggers/cooking/final-model.pt
[rhubarb (0.537205696105957), cranberries (0.5254611968994141), shellfish (0.5021848678588867), lobster (0.5107764601707458), eggs (0.5300827026367188), camping (0.5270642638206482), olive (0.5127106308937073), food-processing (0.5492298603057861), beans (0.5511964559555054), marinade (0.5082960724830627), sour-cream (0.5246407389640808), gas (0.5130353569984436), culinary-uses (0.5102614760398865), zucchini (0.5033143162727356), spanish-cuisine (0.54184889793396), middle-eastern-cuisine (0.5064804553985596), copper-cookware (0.5604353547096252), sifting (0.5457304120063782), heat (0.5082386136054993), cheese (0.5825212597846985), melon (0.5191405415534973), cactus (0.5647401809692383), chicken-breast (0.531735897064209), italian-cuisine (0.5022286772727966), fudge (0.5130940079689026), hot-sauce (0.5414472818374634), tempering (0.5059071183204651), cookbook (0.5138238668441772), turkish-cuisine (0.5154601931